# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from z3 import *
from sklearn.cluster import KMeans
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import sys
import warnings
warnings.filterwarnings("ignore")
import json

# FIXED PARAMETERS

In [2]:
NUM_ZONES = 5

# Clustering

In [3]:
# returns kmeans clusters
def kmeans(X, number_clusters):
    kmeans = KMeans(n_clusters = number_clusters)
    cluster = kmeans.fit(X)
    return cluster

# Convex Hull

In [4]:
# returns convex hull of points associated with zones
def convex_hull(zone, points):
        '''
        Parameters
        ----------
        points : Array
            A set of points.
            
        Returns
        -------
        vertices : Points
            Arranges convex hull vertices in counter-clockwise orientation.
        '''    
        
        hull = ConvexHull(points)
        
        
        #plt.plot(points[:,0], points[:,1], 'o')
        #plt.ylim(0,120)
        #for simplex in hull.simplices:
        #    plt.plot(points[simplex, 0], points[simplex, 1], 'k-')
            
        simplices = hull.simplices
        
        vertices = []
        for index in hull.vertices:
            vertices.append((points[index][0], zone, points[index][1]))
        vertices.append((points[hull.vertices[0]][0], zone, points[hull.vertices[0]][1]))
        
        return vertices

# Cluster Boundary Acquisition

In [5]:
def get_cluster(dataframe, num_clusters):
    list_cluster = []
    
    dataframe = dataframe[dataframe['Day'] <= 15]
    
    for zone in range(NUM_ZONES):
        #plt.figure()
        mod_dataframe = pd.DataFrame()

        for i in range(len(dataframe)):
            if int(dataframe['Occupant\'s Zone'][i] == zone):
                mod_dataframe = mod_dataframe.append(dataframe.loc[i, ['Zone Arrival Time (Minute)', 'Stay Duration (Minute)']])

        features = np.empty([len(mod_dataframe), 2])
        features[:, 0:1] = mod_dataframe.loc[:, ['Zone Arrival Time (Minute)']].values
        features[:, 1:] = mod_dataframe.loc[:, ['Stay Duration (Minute)']].values
        cluster_model = kmeans(features, num_clusters)
        labels = cluster_model.labels_
        
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        for cluster in range(n_clusters):
            points = []
            vertices = []
            
            for k in range(len(labels)):
                if labels[k] == cluster:
                    points.append(features[k])
            
            if len(points) >= 3:
                try:
                    vertices = convex_hull(zone, points)       
                except:
                    pass
            list_cluster.append({"zone_id": zone, "cluster_id": cluster, "points": vertices})

            list_cluster.append({"zone_id": zone, "cluster_id": cluster, "points": vertices})

    return list_cluster

# Clustering for House-A and Occupant-1

In [6]:
cleaned_duration_dataframe_house_A_occ_1 = pd.read_csv("../../data/cleaned/Cleaned-Dataframe_House-A_Occupant-1.csv")
cleaned_duration_dataframe_house_A_occ_1
clusters_house_A_occ_1 = get_cluster(cleaned_duration_dataframe_house_A_occ_1, 29)
# saving clusters to file
with open('../../data/shatter/Clusters-House_A_Occ-1.txt', 'w') as f:
    f.write(json.dumps(clusters_house_A_occ_1))

# Clustering for House-A and Occupant-2

In [7]:
cleaned_duration_dataframe_house_A_occ_2 = pd.read_csv("../../data/cleaned/Cleaned-Dataframe_House-A_Occupant-2.csv")
cleaned_duration_dataframe_house_A_occ_2
clusters_house_A_occ_2 = get_cluster(cleaned_duration_dataframe_house_A_occ_2, 15)
# saving clusters to file
with open('../../data/shatter/Clusters-House_A_Occ-2.txt', 'w') as f:
    f.write(json.dumps(clusters_house_A_occ_2))

# Clustering for House-B and Occupant-1

In [8]:
cleaned_duration_dataframe_house_B_occ_1 = pd.read_csv("../../data/cleaned/Cleaned-Dataframe_House-B_Occupant-1.csv")
cleaned_duration_dataframe_house_B_occ_1
clusters_house_B_occ_1 = get_cluster(cleaned_duration_dataframe_house_B_occ_1, 14)
# saving clusters to file
with open('../../data/shatter/Clusters-House_B_Occ-1.txt', 'w') as f:
    f.write(json.dumps(clusters_house_B_occ_1))

# Clustering for House-B and Occupant-2

In [6]:
cleaned_duration_dataframe_house_B_occ_2 = pd.read_csv("../../data/cleaned/Cleaned-Dataframe_House-B_Occupant-2.csv")
cleaned_duration_dataframe_house_B_occ_2
clusters_house_B_occ_2 = get_cluster(cleaned_duration_dataframe_house_B_occ_2, 10)
# saving clusters to file
with open('../../data/shatter/Clusters-House_B_Occ-2.txt', 'w') as f:
    f.write(json.dumps(clusters_house_B_occ_2))

# Range Claculation Function

In [8]:
def is_left( x, y, init_x, init_y, final_x, final_y):
     return ((final_x - init_x)*(y - init_y) - (final_y - init_y)*(x - init_x)) >= 0

def range_calculation(list_cluster):
    num_timeslots = 1440
    list_time_min = [[[] for j in range(num_timeslots)] for i in range(NUM_ZONES + 1)]
    list_time_max = [[[] for j in range(num_timeslots)] for i in range(NUM_ZONES + 1)]

    for i in range(len(list_cluster)):
        zone_id = list_cluster[i]["zone_id"]
        min_x_range = 1440
        max_x_range = 0

        ##################################################################
        ##################### Zone Constraints ###########################
        ##################################################################
        x = Int('x')
        y = Int('y')

        points = list_cluster[i]["points"]
        zone_constraints = []

        and_constraints = []
        for j in range(len(points) - 1):
            and_constraints.append(is_left(x, y, points[j][0], points[j][2], points[j + 1][0], points[j + 1][2]))

        zone_constraints.append(And(and_constraints))

        #print(zone_constraints)
        ####### Minimum value of X range #######
        o = Optimize()
        o.add(zone_constraints)
        o.minimize(x)
        o.check()
        
        min_x_range = int(str(o.model()[x]))

        ####### Maximum value of X range #######
        o = Optimize()
        o.add(zone_constraints)
        o.maximize(x)
        o.check()
        #print(o.model()[x])

        max_x_range = int(str(o.model()[x]))
        
        
        for j in range(min_x_range, max_x_range):
            ####### Minimum value of Y range #######
            o = Optimize()
            o.add(zone_constraints)
            o.add(x == j)
            o.minimize(y)
            o.check()

            min_y_range = o.model()[y]
            if min_y_range == None:
                min_y_range = 0

            ####### Maximum value of Y range #######
            o = Optimize()
            o.add(zone_constraints)
            o.add(x == j)
            o.maximize(y)
            o.check()

            max_y_range = o.model()[y]
            if max_y_range == None:
                max_y_range = 0
                
            list_time_min[zone_id][j].append(int(str(min_y_range)))
            list_time_max[zone_id][j].append(int(str(max_y_range)))
    return list_time_min, list_time_max

# Attack Scheduling Function

In [11]:
def attack_scheduling(list_cluster, num_timeslots, start_time, final_time):
    list_time_min, list_time_max = range_calculation(list_cluster)
    num_timeslots = interval = 10

    prev_stay = 1
    prev_schedule = -1
    ultimate_cost = 0
    final_schedule = schedule = np.zeros((final_time - start_time))

    for init_time in range(start_time, final_time, interval):
        cost = [Int( 'cost_' + str(i)) for i in range(NUM_ZONES)]
        zones = [Int( 'zones_' + str(i)) for i in range(NUM_ZONES)]

        schedule = [Int( 'schedule_' + str(i)) for i in range(init_time, init_time + interval)]
        stay = [Int( 'stay_' + str(i)) for i in range(num_timeslots)]
        slot_cost = [Int( 'slot_cost_' + str(i)) for i in range(num_timeslots)]

        total_cost = Int('total_cost')

        o = Optimize()
        o.add(cost[0] == 0)
        o.add(cost[1] == 1)
        o.add(cost[2] == 2)
        o.add(cost[3] == 4)
        o.add(cost[4] == 3)


        ############################################################################
        ################## schedule should be withing a valid zone #################
        ############################################################################
        for t in range(len(schedule)):
            or_constraints = []
            for z in range(NUM_ZONES):
                or_constraints.append(schedule[t] == z)
            o.add(Or(or_constraints))

        ###############################################################################################################
        ################## if zone stay threshdold in current time is 0, do not schedule to that zone #################
        ###############################################################################################################

        # base case
        for z in range(1, NUM_ZONES):
            if list_time_min[z][init_time] == [] or (len(list_time_max[z][init_time]) == 1 and list_time_max[z][init_time][0] == 0):
                o.add(Implies(schedule[0] != prev_schedule, schedule[t] != z))

        for t in range(1, len(schedule)):
            for z in range(1, NUM_ZONES):
                if list_time_min[z][init_time + t] == [] or (len(list_time_max[z][init_time + t]) == 1 and list_time_max[z][init_time + t][0] == 0):
                    o.add(Implies(schedule[t] != schedule[t - 1], schedule[t] != z))

        #######################################################################################################################
        ############################################ constraints of stay ######################################################
        #######################################################################################################################
        ######## base case for time 0 ############
        if init_time == 0:
            o.add(stay[0] == 1)
        else:
            o.add(Implies(schedule[0] == prev_schedule, stay[0] == prev_stay + 1))
            o.add(Implies(Not(schedule[0] == prev_schedule), stay[0] == 1))

        for t in range(1, len(schedule)):
            continue_staying = (schedule[t] == schedule[t - 1])
            increment_stay = (stay[t] == stay[t - 1] + 1)
            reset_stay = (stay[t] == 1)

            o.add(Implies(continue_staying, increment_stay))
            o.add(Implies(Not(continue_staying), reset_stay))

        #######################################################################################################################
        ############ move to a zone different that previous timeslot if stay > max threshold in previous timeslot #############
        #######################################################################################################################
        ######## base case for time 0 ############
        o.add(Implies(prev_stay == max(list_time_max[prev_schedule][init_time - prev_stay], default=0), schedule[0] != prev_schedule))

        for t in range(1, len(schedule)):
            max_stay_threshold = 0
            for z in range(1, NUM_ZONES):
                for p_t in range(1, init_time + len(schedule)):
                    continue_staying = (schedule[t] == schedule[t - 1])
                    o.add(Implies(And(schedule[t - 1] == z, stay[t - 1] == p_t, p_t == max(list_time_max[z][init_time + t - p_t], default=0)), Not(continue_staying)))

        #######################################################################################################################
        # must stay in the zone same as the previous timeslot if stay < max && stay is in previous timeslot is out of cluster #
        #######################################################################################################################
        ######## base case for time 0 ############
        ranges_stay_constraints = []
        for k in range(len(list_time_min[prev_schedule][init_time - prev_stay])):
            ranges_stay_constraints.append(And(prev_stay >= list_time_min[prev_schedule][init_time - prev_stay][k], prev_stay <= list_time_max[prev_schedule][init_time - prev_stay][k]))            

        if init_time != 0:
            o.add(Implies(Not(Or(ranges_stay_constraints)), schedule[0] == prev_schedule))

        for t in range(1, len(schedule)):
            for z in range(1, NUM_ZONES):
                for p_t in range(1, init_time + t + 1):
                    ranges_stay_constraints = []
                    for k in range(len(list_time_min[z][init_time + t - p_t])):
                        ranges_stay_constraints.append(And(p_t >= list_time_min[z][init_time + t - p_t][k], p_t <= list_time_max[z][init_time + t - p_t][k]))            

                    continue_staying = (schedule[t] == schedule[t - 1])
                    o.add(Implies(And(schedule[t - 1] == z, stay[t - 1] == p_t, Not(Or(ranges_stay_constraints))), continue_staying))

        for t in range(len(schedule)):
            for z in range(NUM_ZONES):
                o.add(Implies(schedule[t] == z, slot_cost[t] == cost[z]))

        o.add(total_cost == Sum(slot_cost))

        #o.add(schedule[0] == 3)

        o.maximize(total_cost)

        print(o.check())
        print(init_time, o.model()[total_cost])
        ultimate_cost += int(str(o.model()[total_cost]))

        print("Ulimate Cost", init_time, ultimate_cost)

        print("Num_timeslot", num_timeslots, "Optimal_cost", o.model()[total_cost])

        for t in range(num_timeslots):
            print("schedule_" + str(init_time + t), o.model()[schedule[t]])
            final_schedule[init_time + t] = int(str(o.model()[schedule[t]]))

        for t in range(num_timeslots):
            print("stay_" + str(init_time + t), o.model()[stay[t]])

        prev_schedule = int(str(o.model()[schedule[-1]]))
        prev_stay = int(str(o.model()[stay[-1]]))
    return final_schedule

# Greedy Scheduling Function

In [9]:
def greedy_schedule(list_cluster, num_timeslots):
    current_timeslot = 0
    final_timeslot = 0

    alternate_schedule = np.zeros((num_timeslots))
    list_time_min, list_time_max = range_calculation(list_cluster)
    while final_timeslot <= 1439:
        for zone in [3, 4, 2, 1, 0]:
            max_list = list_time_max[zone][current_timeslot]
            for m in range(len(max_list)):
                max_list[m] = int(str(max_list[m]))
            if max_list == []:
                if zone == 0:
                    print("Critical", current_timeslot)
                    final_timeslot = current_timeslot + 1
                    alternate_schedule[current_timeslot: final_timeslot] = 0
                    current_timeslot = final_timeslot
                continue
            max_val = max(max_list)
            if max_val == 0:
                final_timeslot = current_timeslot + 1
                alternate_schedule[current_timeslot: final_timeslot] = 0
                current_timeslot = final_timeslot
            if max_val != 0:
                final_timeslot = current_timeslot + max_val
                print("Here", current_timeslot, final_timeslot)
                alternate_schedule[current_timeslot: final_timeslot] = zone
                print("selected zone", zone)
                current_timeslot = final_timeslot
                break
    return alternate_schedule

# Variable Parameters

In [13]:
num_timeslots = interval = 10
start_time = 0
final_time = 1440

# House-A Occupant-1 Attack Schedule

In [14]:
schedule_house_A_occ_1 = attack_scheduling(clusters_house_A_occ_1, num_timeslots, start_time, final_time)
pd.DataFrame(schedule_house_A_occ_1, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/Partial_KMeans_House-A_Occ-1_SHATTER_Schedule.csv", index = False)

sat
0 40
Ulimate Cost 0 40
Num_timeslot 10 Optimal_cost 40
schedule_0 3
schedule_1 3
schedule_2 3
schedule_3 3
schedule_4 3
schedule_5 3
schedule_6 3
schedule_7 3
schedule_8 3
schedule_9 3
stay_0 1
stay_1 2
stay_2 3
stay_3 4
stay_4 5
stay_5 6
stay_6 7
stay_7 8
stay_8 9
stay_9 10
sat
10 36
Ulimate Cost 10 76
Num_timeslot 10 Optimal_cost 36
schedule_10 3
schedule_11 3
schedule_12 3
schedule_13 0
schedule_14 3
schedule_15 3
schedule_16 3
schedule_17 3
schedule_18 3
schedule_19 3
stay_10 11
stay_11 12
stay_12 13
stay_13 1
stay_14 1
stay_15 2
stay_16 3
stay_17 4
stay_18 5
stay_19 6
sat
20 22
Ulimate Cost 20 98
Num_timeslot 10 Optimal_cost 22
schedule_20 3
schedule_21 2
schedule_22 2
schedule_23 2
schedule_24 2
schedule_25 2
schedule_26 2
schedule_27 2
schedule_28 2
schedule_29 2
stay_20 7
stay_21 1
stay_22 2
stay_23 3
stay_24 4
stay_25 5
stay_26 6
stay_27 7
stay_28 8
stay_29 9
sat
30 20
Ulimate Cost 30 118
Num_timeslot 10 Optimal_cost 20
schedule_30 2
schedule_31 2
schedule_32 2
schedule_33

sat
260 10
Ulimate Cost 260 483
Num_timeslot 10 Optimal_cost 10
schedule_260 1
schedule_261 1
schedule_262 1
schedule_263 1
schedule_264 1
schedule_265 1
schedule_266 1
schedule_267 1
schedule_268 1
schedule_269 1
stay_260 103
stay_261 104
stay_262 105
stay_263 106
stay_264 107
stay_265 108
stay_266 109
stay_267 110
stay_268 111
stay_269 112
sat
270 10
Ulimate Cost 270 493
Num_timeslot 10 Optimal_cost 10
schedule_270 1
schedule_271 1
schedule_272 1
schedule_273 1
schedule_274 1
schedule_275 1
schedule_276 1
schedule_277 1
schedule_278 1
schedule_279 1
stay_270 113
stay_271 114
stay_272 115
stay_273 116
stay_274 117
stay_275 118
stay_276 119
stay_277 120
stay_278 121
stay_279 122
sat
280 10
Ulimate Cost 280 503
Num_timeslot 10 Optimal_cost 10
schedule_280 1
schedule_281 1
schedule_282 1
schedule_283 1
schedule_284 1
schedule_285 1
schedule_286 1
schedule_287 1
schedule_288 1
schedule_289 1
stay_280 123
stay_281 124
stay_282 125
stay_283 126
stay_284 127
stay_285 128
stay_286 129
stay_28

sat
500 10
Ulimate Cost 500 723
Num_timeslot 10 Optimal_cost 10
schedule_500 1
schedule_501 1
schedule_502 1
schedule_503 1
schedule_504 1
schedule_505 1
schedule_506 1
schedule_507 1
schedule_508 1
schedule_509 1
stay_500 343
stay_501 344
stay_502 345
stay_503 346
stay_504 347
stay_505 348
stay_506 349
stay_507 350
stay_508 351
stay_509 352
sat
510 10
Ulimate Cost 510 733
Num_timeslot 10 Optimal_cost 10
schedule_510 1
schedule_511 1
schedule_512 1
schedule_513 1
schedule_514 1
schedule_515 1
schedule_516 1
schedule_517 1
schedule_518 1
schedule_519 1
stay_510 353
stay_511 354
stay_512 355
stay_513 356
stay_514 357
stay_515 358
stay_516 359
stay_517 360
stay_518 361
stay_519 362
sat
520 10
Ulimate Cost 520 743
Num_timeslot 10 Optimal_cost 10
schedule_520 1
schedule_521 1
schedule_522 1
schedule_523 1
schedule_524 1
schedule_525 1
schedule_526 1
schedule_527 1
schedule_528 1
schedule_529 1
stay_520 363
stay_521 364
stay_522 365
stay_523 366
stay_524 367
stay_525 368
stay_526 369
stay_52

sat
750 20
Ulimate Cost 750 1364
Num_timeslot 10 Optimal_cost 20
schedule_750 2
schedule_751 2
schedule_752 2
schedule_753 2
schedule_754 2
schedule_755 2
schedule_756 2
schedule_757 2
schedule_758 2
schedule_759 2
stay_750 2
stay_751 3
stay_752 4
stay_753 5
stay_754 6
stay_755 7
stay_756 8
stay_757 9
stay_758 10
stay_759 11
sat
760 20
Ulimate Cost 760 1384
Num_timeslot 10 Optimal_cost 20
schedule_760 2
schedule_761 2
schedule_762 2
schedule_763 2
schedule_764 2
schedule_765 2
schedule_766 2
schedule_767 2
schedule_768 2
schedule_769 2
stay_760 12
stay_761 13
stay_762 14
stay_763 15
stay_764 16
stay_765 17
stay_766 18
stay_767 19
stay_768 20
stay_769 21
sat
770 20
Ulimate Cost 770 1404
Num_timeslot 10 Optimal_cost 20
schedule_770 2
schedule_771 2
schedule_772 2
schedule_773 2
schedule_774 2
schedule_775 2
schedule_776 2
schedule_777 2
schedule_778 2
schedule_779 2
stay_770 22
stay_771 23
stay_772 24
stay_773 25
stay_774 26
stay_775 27
stay_776 28
stay_777 29
stay_778 30
stay_779 31
sat

sat
1000 37
Ulimate Cost 1000 2000
Num_timeslot 10 Optimal_cost 37
schedule_1000 2
schedule_1001 4
schedule_1002 3
schedule_1003 3
schedule_1004 3
schedule_1005 3
schedule_1006 3
schedule_1007 3
schedule_1008 3
schedule_1009 3
stay_1000 16
stay_1001 1
stay_1002 1
stay_1003 2
stay_1004 3
stay_1005 4
stay_1006 5
stay_1007 6
stay_1008 7
stay_1009 8
sat
1010 40
Ulimate Cost 1010 2040
Num_timeslot 10 Optimal_cost 40
schedule_1010 3
schedule_1011 3
schedule_1012 3
schedule_1013 3
schedule_1014 3
schedule_1015 3
schedule_1016 3
schedule_1017 3
schedule_1018 3
schedule_1019 3
stay_1010 9
stay_1011 10
stay_1012 11
stay_1013 12
stay_1014 13
stay_1015 14
stay_1016 15
stay_1017 16
stay_1018 17
stay_1019 18
sat
1020 40
Ulimate Cost 1020 2080
Num_timeslot 10 Optimal_cost 40
schedule_1020 3
schedule_1021 3
schedule_1022 3
schedule_1023 3
schedule_1024 3
schedule_1025 3
schedule_1026 3
schedule_1027 3
schedule_1028 3
schedule_1029 3
stay_1020 19
stay_1021 20
stay_1022 21
stay_1023 22
stay_1024 23
stay

sat
1240 40
Ulimate Cost 1240 2671
Num_timeslot 10 Optimal_cost 40
schedule_1240 3
schedule_1241 3
schedule_1242 3
schedule_1243 3
schedule_1244 3
schedule_1245 3
schedule_1246 3
schedule_1247 3
schedule_1248 3
schedule_1249 3
stay_1240 28
stay_1241 29
stay_1242 30
stay_1243 31
stay_1244 32
stay_1245 33
stay_1246 34
stay_1247 35
stay_1248 36
stay_1249 37
sat
1250 40
Ulimate Cost 1250 2711
Num_timeslot 10 Optimal_cost 40
schedule_1250 3
schedule_1251 3
schedule_1252 3
schedule_1253 3
schedule_1254 3
schedule_1255 3
schedule_1256 3
schedule_1257 3
schedule_1258 3
schedule_1259 3
stay_1250 38
stay_1251 39
stay_1252 40
stay_1253 41
stay_1254 42
stay_1255 43
stay_1256 44
stay_1257 45
stay_1258 46
stay_1259 47
sat
1260 20
Ulimate Cost 1260 2731
Num_timeslot 10 Optimal_cost 20
schedule_1260 3
schedule_1261 0
schedule_1262 2
schedule_1263 2
schedule_1264 2
schedule_1265 2
schedule_1266 2
schedule_1267 2
schedule_1268 2
schedule_1269 2
stay_1260 48
stay_1261 1
stay_1262 1
stay_1263 2
stay_1264 

# House-A Occupant-1 Greedy Schedule

In [15]:
greedy_schedule_house_A_occ_1 = greedy_schedule(clusters_house_A_occ_1, 1440)
pd.DataFrame(greedy_schedule_house_A_occ_1, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/Partial_KMeans_House-A_Occ-1_Greedy_Schedule.csv", index = False)

Here 0 18
selected zone 3
Here 18 22
selected zone 3
Here 22 146
selected zone 2
Here 146 151
selected zone 4
Here 151 157
selected zone 4
Here 157 164
selected zone 0
Here 164 584
selected zone 1
Here 584 588
selected zone 4
Here 588 591
selected zone 4
Here 591 594
selected zone 4
Here 594 596
selected zone 4
Here 596 634
selected zone 2
Here 634 712
selected zone 3
Here 712 729
selected zone 3
Here 729 748
selected zone 4
Here 748 762
selected zone 4
Here 762 766
selected zone 4
Here 766 920
selected zone 2
Here 920 939
selected zone 2
Here 939 954
selected zone 3
Critical 954
Here 955 985
selected zone 3
Here 985 1008
selected zone 2
Here 1008 1048
selected zone 3
Here 1048 1049
selected zone 4
Here 1049 1050
selected zone 4
Here 1050 1052
selected zone 4
Here 1052 1053
selected zone 4
Here 1053 1054
selected zone 4
Here 1054 1055
selected zone 4
Here 1055 1056
selected zone 4
Here 1056 1057
selected zone 4
Here 1057 1058
selected zone 4
Here 1058 1059
selected zone 4
Here 1059 106

# House-A Occupant-2 Attack Schedule

In [16]:
schedule_house_A_occ_2 = attack_scheduling(clusters_house_A_occ_2, num_timeslots, start_time, final_time)
pd.DataFrame(schedule_house_A_occ_2, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/Partial_KMeans_House-A_Occ-2_SHATTER_Schedule.csv", index = False)

sat
0 22
Ulimate Cost 0 22
Num_timeslot 10 Optimal_cost 22
schedule_0 2
schedule_1 2
schedule_2 2
schedule_3 2
schedule_4 2
schedule_5 2
schedule_6 4
schedule_7 4
schedule_8 2
schedule_9 2
stay_0 1
stay_1 2
stay_2 3
stay_3 4
stay_4 5
stay_5 6
stay_6 1
stay_7 2
stay_8 1
stay_9 2
sat
10 27
Ulimate Cost 10 49
Num_timeslot 10 Optimal_cost 27
schedule_10 2
schedule_11 2
schedule_12 2
schedule_13 4
schedule_14 4
schedule_15 4
schedule_16 4
schedule_17 4
schedule_18 4
schedule_19 4
stay_10 3
stay_11 4
stay_12 5
stay_13 1
stay_14 2
stay_15 3
stay_16 4
stay_17 5
stay_18 6
stay_19 7
sat
20 27
Ulimate Cost 20 76
Num_timeslot 10 Optimal_cost 27
schedule_20 4
schedule_21 4
schedule_22 4
schedule_23 0
schedule_24 4
schedule_25 4
schedule_26 4
schedule_27 4
schedule_28 4
schedule_29 4
stay_20 8
stay_21 9
stay_22 10
stay_23 1
stay_24 1
stay_25 2
stay_26 3
stay_27 4
stay_28 5
stay_29 6
sat
30 30
Ulimate Cost 30 106
Num_timeslot 10 Optimal_cost 30
schedule_30 4
schedule_31 4
schedule_32 4
schedule_33 4


sat
260 10
Ulimate Cost 260 610
Num_timeslot 10 Optimal_cost 10
schedule_260 1
schedule_261 1
schedule_262 1
schedule_263 1
schedule_264 1
schedule_265 1
schedule_266 1
schedule_267 1
schedule_268 1
schedule_269 1
stay_260 90
stay_261 91
stay_262 92
stay_263 93
stay_264 94
stay_265 95
stay_266 96
stay_267 97
stay_268 98
stay_269 99
sat
270 10
Ulimate Cost 270 620
Num_timeslot 10 Optimal_cost 10
schedule_270 1
schedule_271 1
schedule_272 1
schedule_273 1
schedule_274 1
schedule_275 1
schedule_276 1
schedule_277 1
schedule_278 1
schedule_279 1
stay_270 100
stay_271 101
stay_272 102
stay_273 103
stay_274 104
stay_275 105
stay_276 106
stay_277 107
stay_278 108
stay_279 109
sat
280 10
Ulimate Cost 280 630
Num_timeslot 10 Optimal_cost 10
schedule_280 1
schedule_281 1
schedule_282 1
schedule_283 1
schedule_284 1
schedule_285 1
schedule_286 1
schedule_287 1
schedule_288 1
schedule_289 1
stay_280 110
stay_281 111
stay_282 112
stay_283 113
stay_284 114
stay_285 115
stay_286 116
stay_287 117
stay

sat
500 30
Ulimate Cost 500 876
Num_timeslot 10 Optimal_cost 30
schedule_500 4
schedule_501 4
schedule_502 4
schedule_503 4
schedule_504 4
schedule_505 4
schedule_506 4
schedule_507 4
schedule_508 4
schedule_509 4
stay_500 4
stay_501 5
stay_502 6
stay_503 7
stay_504 8
stay_505 9
stay_506 10
stay_507 11
stay_508 12
stay_509 13
sat
510 30
Ulimate Cost 510 906
Num_timeslot 10 Optimal_cost 30
schedule_510 4
schedule_511 4
schedule_512 4
schedule_513 4
schedule_514 4
schedule_515 4
schedule_516 4
schedule_517 4
schedule_518 4
schedule_519 4
stay_510 14
stay_511 15
stay_512 16
stay_513 17
stay_514 18
stay_515 19
stay_516 20
stay_517 21
stay_518 22
stay_519 23
sat
520 28
Ulimate Cost 520 934
Num_timeslot 10 Optimal_cost 28
schedule_520 4
schedule_521 4
schedule_522 4
schedule_523 4
schedule_524 4
schedule_525 4
schedule_526 4
schedule_527 4
schedule_528 4
schedule_529 1
stay_520 24
stay_521 25
stay_522 26
stay_523 27
stay_524 28
stay_525 29
stay_526 30
stay_527 31
stay_528 32
stay_529 1
sat
5

sat
750 0
Ulimate Cost 750 1385
Num_timeslot 10 Optimal_cost 0
schedule_750 0
schedule_751 0
schedule_752 0
schedule_753 0
schedule_754 0
schedule_755 0
schedule_756 0
schedule_757 0
schedule_758 0
schedule_759 0
stay_750 33
stay_751 34
stay_752 35
stay_753 36
stay_754 37
stay_755 38
stay_756 39
stay_757 40
stay_758 41
stay_759 42
sat
760 0
Ulimate Cost 760 1385
Num_timeslot 10 Optimal_cost 0
schedule_760 0
schedule_761 0
schedule_762 0
schedule_763 0
schedule_764 0
schedule_765 0
schedule_766 0
schedule_767 0
schedule_768 0
schedule_769 0
stay_760 43
stay_761 44
stay_762 45
stay_763 46
stay_764 47
stay_765 48
stay_766 49
stay_767 50
stay_768 51
stay_769 52
sat
770 0
Ulimate Cost 770 1385
Num_timeslot 10 Optimal_cost 0
schedule_770 0
schedule_771 0
schedule_772 0
schedule_773 0
schedule_774 0
schedule_775 0
schedule_776 0
schedule_777 0
schedule_778 0
schedule_779 0
stay_770 53
stay_771 54
stay_772 55
stay_773 56
stay_774 57
stay_775 58
stay_776 59
stay_777 60
stay_778 61
stay_779 62
s

sat
1000 11
Ulimate Cost 1000 1766
Num_timeslot 10 Optimal_cost 11
schedule_1000 1
schedule_1001 1
schedule_1002 1
schedule_1003 1
schedule_1004 1
schedule_1005 1
schedule_1006 1
schedule_1007 0
schedule_1008 2
schedule_1009 2
stay_1000 35
stay_1001 36
stay_1002 37
stay_1003 38
stay_1004 39
stay_1005 40
stay_1006 41
stay_1007 1
stay_1008 1
stay_1009 2
sat
1010 20
Ulimate Cost 1010 1786
Num_timeslot 10 Optimal_cost 20
schedule_1010 2
schedule_1011 2
schedule_1012 2
schedule_1013 2
schedule_1014 2
schedule_1015 2
schedule_1016 2
schedule_1017 2
schedule_1018 2
schedule_1019 2
stay_1010 3
stay_1011 4
stay_1012 5
stay_1013 6
stay_1014 7
stay_1015 8
stay_1016 9
stay_1017 10
stay_1018 11
stay_1019 12
sat
1020 20
Ulimate Cost 1020 1806
Num_timeslot 10 Optimal_cost 20
schedule_1020 2
schedule_1021 2
schedule_1022 2
schedule_1023 2
schedule_1024 2
schedule_1025 2
schedule_1026 2
schedule_1027 2
schedule_1028 2
schedule_1029 2
stay_1020 13
stay_1021 14
stay_1022 15
stay_1023 16
stay_1024 17
stay

sat
1240 18
Ulimate Cost 1240 2292
Num_timeslot 10 Optimal_cost 18
schedule_1240 2
schedule_1241 2
schedule_1242 2
schedule_1243 2
schedule_1244 2
schedule_1245 2
schedule_1246 0
schedule_1247 0
schedule_1248 4
schedule_1249 4
stay_1240 10
stay_1241 11
stay_1242 12
stay_1243 13
stay_1244 14
stay_1245 15
stay_1246 1
stay_1247 2
stay_1248 1
stay_1249 2
sat
1250 18
Ulimate Cost 1250 2310
Num_timeslot 10 Optimal_cost 18
schedule_1250 0
schedule_1251 4
schedule_1252 4
schedule_1253 0
schedule_1254 0
schedule_1255 4
schedule_1256 4
schedule_1257 0
schedule_1258 4
schedule_1259 4
stay_1250 1
stay_1251 1
stay_1252 2
stay_1253 1
stay_1254 2
stay_1255 1
stay_1256 2
stay_1257 1
stay_1258 1
stay_1259 2
sat
1260 21
Ulimate Cost 1260 2331
Num_timeslot 10 Optimal_cost 21
schedule_1260 0
schedule_1261 4
schedule_1262 4
schedule_1263 0
schedule_1264 4
schedule_1265 4
schedule_1266 4
schedule_1267 0
schedule_1268 4
schedule_1269 4
stay_1260 1
stay_1261 1
stay_1262 2
stay_1263 1
stay_1264 1
stay_1265 2
s

# House-A Occupant-2 Greedy Schedule

In [17]:
greedy_schedule_house_A_occ_2 = greedy_schedule(clusters_house_A_occ_2, 1440)
pd.DataFrame(greedy_schedule_house_A_occ_2, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/Partial_KMeans_House-A_Occ-2_Greedy_Schedule.csv", index = False)

Here 0 41
selected zone 2
Here 41 69
selected zone 4
Here 69 438
selected zone 1
Here 438 440
selected zone 0
Here 440 443
selected zone 0
Here 443 447
selected zone 0
Here 447 452
selected zone 0
Here 452 458
selected zone 0
Here 458 466
selected zone 0
Here 466 508
selected zone 4
Here 508 519
selected zone 1
Here 519 540
selected zone 4
Here 540 564
selected zone 4
Here 564 576
selected zone 2
Here 576 595
selected zone 4
Here 595 615
selected zone 4
Here 615 649
selected zone 2
Here 649 697
selected zone 2
Critical 697
Critical 698
Critical 699
Critical 700
Critical 701
Critical 702
Critical 703
Critical 704
Critical 705
Critical 706
Critical 707
Critical 708
Critical 709
Critical 710
Critical 711
Critical 712
Critical 713
Critical 714
Critical 715
Critical 716
Critical 717
Critical 718
Critical 719
Critical 720
Critical 721
Critical 722
Critical 723
Critical 724
Critical 725
Critical 726
Critical 727
Critical 728
Critical 729
Critical 730
Critical 731
Critical 732
Critical 733
Cri

# House-B Occupant-1 Attack Schedule

In [18]:
schedule_house_B_occ_1 = attack_scheduling(clusters_house_B_occ_1, num_timeslots, start_time, final_time)
pd.DataFrame(schedule_house_B_occ_1, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/Partial_KMeans_House-B_Occ-1_SHATTER_Schedule.csv", index = False)

sat
0 20
Ulimate Cost 0 20
Num_timeslot 10 Optimal_cost 20
schedule_0 2
schedule_1 2
schedule_2 2
schedule_3 2
schedule_4 2
schedule_5 2
schedule_6 2
schedule_7 2
schedule_8 2
schedule_9 2
stay_0 1
stay_1 2
stay_2 3
stay_3 4
stay_4 5
stay_5 6
stay_6 7
stay_7 8
stay_8 9
stay_9 10
sat
10 20
Ulimate Cost 10 40
Num_timeslot 10 Optimal_cost 20
schedule_10 2
schedule_11 2
schedule_12 2
schedule_13 2
schedule_14 2
schedule_15 2
schedule_16 2
schedule_17 2
schedule_18 2
schedule_19 2
stay_10 11
stay_11 12
stay_12 13
stay_13 14
stay_14 15
stay_15 16
stay_16 17
stay_17 18
stay_18 19
stay_19 20
sat
20 20
Ulimate Cost 20 60
Num_timeslot 10 Optimal_cost 20
schedule_20 2
schedule_21 2
schedule_22 2
schedule_23 2
schedule_24 2
schedule_25 2
schedule_26 2
schedule_27 2
schedule_28 2
schedule_29 2
stay_20 21
stay_21 22
stay_22 23
stay_23 24
stay_24 25
stay_25 26
stay_26 27
stay_27 28
stay_28 29
stay_29 30
sat
30 20
Ulimate Cost 30 80
Num_timeslot 10 Optimal_cost 20
schedule_30 2
schedule_31 2
schedule_

sat
260 30
Ulimate Cost 260 743
Num_timeslot 10 Optimal_cost 30
schedule_260 4
schedule_261 4
schedule_262 4
schedule_263 4
schedule_264 4
schedule_265 4
schedule_266 4
schedule_267 4
schedule_268 4
schedule_269 4
stay_260 197
stay_261 198
stay_262 199
stay_263 200
stay_264 201
stay_265 202
stay_266 203
stay_267 204
stay_268 205
stay_269 206
sat
270 30
Ulimate Cost 270 773
Num_timeslot 10 Optimal_cost 30
schedule_270 4
schedule_271 4
schedule_272 4
schedule_273 4
schedule_274 4
schedule_275 4
schedule_276 4
schedule_277 4
schedule_278 4
schedule_279 4
stay_270 207
stay_271 208
stay_272 209
stay_273 210
stay_274 211
stay_275 212
stay_276 213
stay_277 214
stay_278 215
stay_279 216
sat
280 30
Ulimate Cost 280 803
Num_timeslot 10 Optimal_cost 30
schedule_280 4
schedule_281 4
schedule_282 4
schedule_283 4
schedule_284 4
schedule_285 4
schedule_286 4
schedule_287 4
schedule_288 4
schedule_289 4
stay_280 217
stay_281 218
stay_282 219
stay_283 220
stay_284 221
stay_285 222
stay_286 223
stay_28

sat
500 30
Ulimate Cost 500 1463
Num_timeslot 10 Optimal_cost 30
schedule_500 4
schedule_501 4
schedule_502 4
schedule_503 4
schedule_504 4
schedule_505 4
schedule_506 4
schedule_507 4
schedule_508 4
schedule_509 4
stay_500 437
stay_501 438
stay_502 439
stay_503 440
stay_504 441
stay_505 442
stay_506 443
stay_507 444
stay_508 445
stay_509 446
sat
510 30
Ulimate Cost 510 1493
Num_timeslot 10 Optimal_cost 30
schedule_510 4
schedule_511 4
schedule_512 4
schedule_513 4
schedule_514 4
schedule_515 4
schedule_516 4
schedule_517 4
schedule_518 4
schedule_519 4
stay_510 447
stay_511 448
stay_512 449
stay_513 450
stay_514 451
stay_515 452
stay_516 453
stay_517 454
stay_518 455
stay_519 456
sat
520 30
Ulimate Cost 520 1523
Num_timeslot 10 Optimal_cost 30
schedule_520 4
schedule_521 4
schedule_522 4
schedule_523 4
schedule_524 4
schedule_525 4
schedule_526 4
schedule_527 4
schedule_528 4
schedule_529 4
stay_520 457
stay_521 458
stay_522 459
stay_523 460
stay_524 461
stay_525 462
stay_526 463
stay

sat
740 30
Ulimate Cost 740 2183
Num_timeslot 10 Optimal_cost 30
schedule_740 4
schedule_741 4
schedule_742 4
schedule_743 4
schedule_744 4
schedule_745 4
schedule_746 4
schedule_747 4
schedule_748 4
schedule_749 4
stay_740 677
stay_741 678
stay_742 679
stay_743 680
stay_744 681
stay_745 682
stay_746 683
stay_747 684
stay_748 685
stay_749 686
sat
750 30
Ulimate Cost 750 2213
Num_timeslot 10 Optimal_cost 30
schedule_750 4
schedule_751 4
schedule_752 4
schedule_753 4
schedule_754 4
schedule_755 4
schedule_756 4
schedule_757 4
schedule_758 4
schedule_759 4
stay_750 687
stay_751 688
stay_752 689
stay_753 690
stay_754 691
stay_755 692
stay_756 693
stay_757 694
stay_758 695
stay_759 696
sat
760 30
Ulimate Cost 760 2243
Num_timeslot 10 Optimal_cost 30
schedule_760 4
schedule_761 4
schedule_762 4
schedule_763 4
schedule_764 4
schedule_765 4
schedule_766 4
schedule_767 4
schedule_768 4
schedule_769 4
stay_760 697
stay_761 698
stay_762 699
stay_763 700
stay_764 701
stay_765 702
stay_766 703
stay

sat
980 30
Ulimate Cost 980 2903
Num_timeslot 10 Optimal_cost 30
schedule_980 4
schedule_981 4
schedule_982 4
schedule_983 4
schedule_984 4
schedule_985 4
schedule_986 4
schedule_987 4
schedule_988 4
schedule_989 4
stay_980 917
stay_981 918
stay_982 919
stay_983 920
stay_984 921
stay_985 922
stay_986 923
stay_987 924
stay_988 925
stay_989 926
sat
990 30
Ulimate Cost 990 2933
Num_timeslot 10 Optimal_cost 30
schedule_990 4
schedule_991 4
schedule_992 4
schedule_993 4
schedule_994 4
schedule_995 4
schedule_996 4
schedule_997 4
schedule_998 4
schedule_999 4
stay_990 927
stay_991 928
stay_992 929
stay_993 930
stay_994 931
stay_995 932
stay_996 933
stay_997 934
stay_998 935
stay_999 936
sat
1000 30
Ulimate Cost 1000 2963
Num_timeslot 10 Optimal_cost 30
schedule_1000 4
schedule_1001 4
schedule_1002 4
schedule_1003 4
schedule_1004 4
schedule_1005 4
schedule_1006 4
schedule_1007 4
schedule_1008 4
schedule_1009 4
stay_1000 937
stay_1001 938
stay_1002 939
stay_1003 940
stay_1004 941
stay_1005 942

sat
1210 30
Ulimate Cost 1210 3593
Num_timeslot 10 Optimal_cost 30
schedule_1210 4
schedule_1211 4
schedule_1212 4
schedule_1213 4
schedule_1214 4
schedule_1215 4
schedule_1216 4
schedule_1217 4
schedule_1218 4
schedule_1219 4
stay_1210 1147
stay_1211 1148
stay_1212 1149
stay_1213 1150
stay_1214 1151
stay_1215 1152
stay_1216 1153
stay_1217 1154
stay_1218 1155
stay_1219 1156
sat
1220 30
Ulimate Cost 1220 3623
Num_timeslot 10 Optimal_cost 30
schedule_1220 4
schedule_1221 4
schedule_1222 4
schedule_1223 4
schedule_1224 4
schedule_1225 4
schedule_1226 4
schedule_1227 4
schedule_1228 4
schedule_1229 4
stay_1220 1157
stay_1221 1158
stay_1222 1159
stay_1223 1160
stay_1224 1161
stay_1225 1162
stay_1226 1163
stay_1227 1164
stay_1228 1165
stay_1229 1166
sat
1230 30
Ulimate Cost 1230 3653
Num_timeslot 10 Optimal_cost 30
schedule_1230 4
schedule_1231 4
schedule_1232 4
schedule_1233 4
schedule_1234 4
schedule_1235 4
schedule_1236 4
schedule_1237 4
schedule_1238 4
schedule_1239 4
stay_1230 1167
stay

sat
1430 30
Ulimate Cost 1430 4253
Num_timeslot 10 Optimal_cost 30
schedule_1430 4
schedule_1431 4
schedule_1432 4
schedule_1433 4
schedule_1434 4
schedule_1435 4
schedule_1436 4
schedule_1437 4
schedule_1438 4
schedule_1439 4
stay_1430 1367
stay_1431 1368
stay_1432 1369
stay_1433 1370
stay_1434 1371
stay_1435 1372
stay_1436 1373
stay_1437 1374
stay_1438 1375
stay_1439 1376


# House-B Occupant-1 Greedy Schedule

In [19]:
greedy_schedule_house_B_occ_1 = greedy_schedule(clusters_house_B_occ_1, 1440)
pd.DataFrame(greedy_schedule_house_B_occ_1, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/Partial_KMeans_House-B_Occ-1_Greedy_Schedule.csv", index = False)

Here 0 61
selected zone 2
Here 61 537
selected zone 1
Here 537 540
selected zone 4
Here 540 543
selected zone 4
Here 543 546
selected zone 4
Here 546 549
selected zone 4
Here 549 552
selected zone 4
Here 552 555
selected zone 4
Here 555 558
selected zone 4
Here 558 586
selected zone 2
Here 586 590
selected zone 4
Here 590 593
selected zone 4
Here 593 595
selected zone 4
Here 595 597
selected zone 4
Here 597 598
selected zone 4
Here 598 1388
selected zone 0
Here 1388 1439
selected zone 1
Critical 1439


# House-B Occupant-2 Attack Schedule

In [20]:
schedule_house_B_occ_2 = attack_scheduling(clusters_house_B_occ_2, num_timeslots, start_time, final_time)
pd.DataFrame(schedule_house_B_occ_2, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/Partial_KMeans_House-B_Occ-2_SHATTER_Schedule.csv", index = False)

sat
0 20
Ulimate Cost 0 20
Num_timeslot 10 Optimal_cost 20
schedule_0 2
schedule_1 2
schedule_2 2
schedule_3 2
schedule_4 2
schedule_5 2
schedule_6 2
schedule_7 2
schedule_8 2
schedule_9 2
stay_0 1
stay_1 2
stay_2 3
stay_3 4
stay_4 5
stay_5 6
stay_6 7
stay_7 8
stay_8 9
stay_9 10
sat
10 20
Ulimate Cost 10 40
Num_timeslot 10 Optimal_cost 20
schedule_10 2
schedule_11 2
schedule_12 2
schedule_13 2
schedule_14 2
schedule_15 2
schedule_16 2
schedule_17 2
schedule_18 2
schedule_19 2
stay_10 11
stay_11 12
stay_12 13
stay_13 14
stay_14 15
stay_15 16
stay_16 17
stay_17 18
stay_18 19
stay_19 20
sat
20 20
Ulimate Cost 20 60
Num_timeslot 10 Optimal_cost 20
schedule_20 2
schedule_21 2
schedule_22 2
schedule_23 2
schedule_24 2
schedule_25 2
schedule_26 2
schedule_27 2
schedule_28 2
schedule_29 2
stay_20 21
stay_21 22
stay_22 23
stay_23 24
stay_24 25
stay_25 26
stay_26 27
stay_27 28
stay_28 29
stay_29 30
sat
30 20
Ulimate Cost 30 80
Num_timeslot 10 Optimal_cost 20
schedule_30 2
schedule_31 2
schedule_

sat
260 10
Ulimate Cost 260 401
Num_timeslot 10 Optimal_cost 10
schedule_260 1
schedule_261 1
schedule_262 1
schedule_263 1
schedule_264 1
schedule_265 1
schedule_266 1
schedule_267 1
schedule_268 1
schedule_269 1
stay_260 127
stay_261 128
stay_262 129
stay_263 130
stay_264 131
stay_265 132
stay_266 133
stay_267 134
stay_268 135
stay_269 136
sat
270 10
Ulimate Cost 270 411
Num_timeslot 10 Optimal_cost 10
schedule_270 1
schedule_271 1
schedule_272 1
schedule_273 1
schedule_274 1
schedule_275 1
schedule_276 1
schedule_277 1
schedule_278 1
schedule_279 1
stay_270 137
stay_271 138
stay_272 139
stay_273 140
stay_274 141
stay_275 142
stay_276 143
stay_277 144
stay_278 145
stay_279 146
sat
280 10
Ulimate Cost 280 421
Num_timeslot 10 Optimal_cost 10
schedule_280 1
schedule_281 1
schedule_282 1
schedule_283 1
schedule_284 1
schedule_285 1
schedule_286 1
schedule_287 1
schedule_288 1
schedule_289 1
stay_280 147
stay_281 148
stay_282 149
stay_283 150
stay_284 151
stay_285 152
stay_286 153
stay_28

sat
500 10
Ulimate Cost 500 641
Num_timeslot 10 Optimal_cost 10
schedule_500 1
schedule_501 1
schedule_502 1
schedule_503 1
schedule_504 1
schedule_505 1
schedule_506 1
schedule_507 1
schedule_508 1
schedule_509 1
stay_500 367
stay_501 368
stay_502 369
stay_503 370
stay_504 371
stay_505 372
stay_506 373
stay_507 374
stay_508 375
stay_509 376
sat
510 10
Ulimate Cost 510 651
Num_timeslot 10 Optimal_cost 10
schedule_510 1
schedule_511 1
schedule_512 1
schedule_513 1
schedule_514 1
schedule_515 1
schedule_516 1
schedule_517 1
schedule_518 1
schedule_519 1
stay_510 377
stay_511 378
stay_512 379
stay_513 380
stay_514 381
stay_515 382
stay_516 383
stay_517 384
stay_518 385
stay_519 386
sat
520 20
Ulimate Cost 520 671
Num_timeslot 10 Optimal_cost 20
schedule_520 1
schedule_521 4
schedule_522 4
schedule_523 0
schedule_524 4
schedule_525 4
schedule_526 0
schedule_527 4
schedule_528 4
schedule_529 1
stay_520 387
stay_521 1
stay_522 2
stay_523 1
stay_524 1
stay_525 2
stay_526 1
stay_527 1
stay_528

sat
750 25
Ulimate Cost 750 1213
Num_timeslot 10 Optimal_cost 25
schedule_750 2
schedule_751 2
schedule_752 2
schedule_753 4
schedule_754 4
schedule_755 4
schedule_756 1
schedule_757 4
schedule_758 4
schedule_759 4
stay_750 49
stay_751 50
stay_752 51
stay_753 1
stay_754 2
stay_755 3
stay_756 1
stay_757 1
stay_758 2
stay_759 3
sat
760 24
Ulimate Cost 760 1237
Num_timeslot 10 Optimal_cost 24
schedule_760 1
schedule_761 4
schedule_762 4
schedule_763 4
schedule_764 1
schedule_765 4
schedule_766 4
schedule_767 4
schedule_768 1
schedule_769 4
stay_760 1
stay_761 1
stay_762 2
stay_763 3
stay_764 1
stay_765 1
stay_766 2
stay_767 3
stay_768 1
stay_769 1
sat
770 26
Ulimate Cost 770 1263
Num_timeslot 10 Optimal_cost 26
schedule_770 4
schedule_771 4
schedule_772 1
schedule_773 4
schedule_774 4
schedule_775 4
schedule_776 1
schedule_777 4
schedule_778 4
schedule_779 4
stay_770 2
stay_771 3
stay_772 1
stay_773 1
stay_774 2
stay_775 3
stay_776 1
stay_777 1
stay_778 2
stay_779 3
sat
780 24
Ulimate Cos

sat
1000 0
Ulimate Cost 1000 1343
Num_timeslot 10 Optimal_cost 0
schedule_1000 0
schedule_1001 0
schedule_1002 0
schedule_1003 0
schedule_1004 0
schedule_1005 0
schedule_1006 0
schedule_1007 0
schedule_1008 0
schedule_1009 0
stay_1000 174
stay_1001 175
stay_1002 176
stay_1003 177
stay_1004 178
stay_1005 179
stay_1006 180
stay_1007 181
stay_1008 182
stay_1009 183
sat
1010 0
Ulimate Cost 1010 1343
Num_timeslot 10 Optimal_cost 0
schedule_1010 0
schedule_1011 0
schedule_1012 0
schedule_1013 0
schedule_1014 0
schedule_1015 0
schedule_1016 0
schedule_1017 0
schedule_1018 0
schedule_1019 0
stay_1010 184
stay_1011 185
stay_1012 186
stay_1013 187
stay_1014 188
stay_1015 189
stay_1016 190
stay_1017 191
stay_1018 192
stay_1019 193
sat
1020 0
Ulimate Cost 1020 1343
Num_timeslot 10 Optimal_cost 0
schedule_1020 0
schedule_1021 0
schedule_1022 0
schedule_1023 0
schedule_1024 0
schedule_1025 0
schedule_1026 0
schedule_1027 0
schedule_1028 0
schedule_1029 0
stay_1020 194
stay_1021 195
stay_1022 196
sta

sat
1230 20
Ulimate Cost 1230 1576
Num_timeslot 10 Optimal_cost 20
schedule_1230 2
schedule_1231 2
schedule_1232 2
schedule_1233 2
schedule_1234 2
schedule_1235 2
schedule_1236 2
schedule_1237 2
schedule_1238 2
schedule_1239 2
stay_1230 43
stay_1231 44
stay_1232 45
stay_1233 46
stay_1234 47
stay_1235 48
stay_1236 49
stay_1237 50
stay_1238 51
stay_1239 52
sat
1240 20
Ulimate Cost 1240 1596
Num_timeslot 10 Optimal_cost 20
schedule_1240 2
schedule_1241 2
schedule_1242 2
schedule_1243 2
schedule_1244 2
schedule_1245 2
schedule_1246 2
schedule_1247 2
schedule_1248 2
schedule_1249 2
stay_1240 53
stay_1241 54
stay_1242 55
stay_1243 56
stay_1244 57
stay_1245 58
stay_1246 59
stay_1247 60
stay_1248 61
stay_1249 62
sat
1250 20
Ulimate Cost 1250 1616
Num_timeslot 10 Optimal_cost 20
schedule_1250 2
schedule_1251 2
schedule_1252 2
schedule_1253 2
schedule_1254 2
schedule_1255 2
schedule_1256 2
schedule_1257 2
schedule_1258 2
schedule_1259 2
stay_1250 63
stay_1251 64
stay_1252 65
stay_1253 66
stay_12

# House-B Occupant-2 Greedy Schedule

In [10]:
greedy_schedule_house_B_occ_2 = greedy_schedule(clusters_house_B_occ_2, 1440)
pd.DataFrame(greedy_schedule_house_B_occ_2, columns = ["Schedule"], dtype='int64').to_csv("../../data/shatter/Partial_KMeans_House-B_Occ-2_Greedy_Schedule.csv", index = False)

Here 0 60
selected zone 2
Here 60 90
selected zone 2
Here 90 114
selected zone 2
Here 114 134
selected zone 2
Here 134 411
selected zone 1
Here 411 415
selected zone 4
Here 415 420
selected zone 4
Here 420 426
selected zone 4
Here 426 433
selected zone 4
Here 433 441
selected zone 4
Here 441 450
selected zone 4
Here 450 460
selected zone 4
Here 460 471
selected zone 4
Critical 471
Critical 472
Critical 473
Here 474 577
selected zone 0
Here 577 598
selected zone 4
Here 598 618
selected zone 4
Here 618 638
selected zone 4
Here 638 687
selected zone 2
Here 687 697
selected zone 4
Here 697 709
selected zone 4
Here 709 775
selected zone 2
Here 775 778
selected zone 4
Here 778 781
selected zone 4
Here 781 784
selected zone 4
Here 784 787
selected zone 4
Here 787 790
selected zone 4
Here 790 793
selected zone 4
Here 793 796
selected zone 4
Here 796 799
selected zone 4
Here 799 802
selected zone 4
Here 802 829
selected zone 2
Critical 829
Critical 830
Critical 831
Critical 832
Critical 833
Cri